## Tutorial 20 - Coanda Effect
**_Keywords: bifurcation problem, non-uniqueness, Navier-Stokes equations, supremizer operator_**

### 1. Introduction

In this tutorial, we will study the Coanda Effect held by the Navier-Stokes equations over a two-dimensional sudden-expansion channel $\Omega$ shown below:

<img src="https://github.com/RBniCS/RBniCS/raw/master/tutorials/20_coanda_effect/data/channel.png" width="80%"/>

A Poiseuille flow profile is imposed on the inlet boundary, and a no-flow (zero velocity) condition is imposed on the walls. A homogeneous Neumann condition of the Cauchy stress tensor is applied at the outflow boundary.

The inflow velocity boundary condition is characterized by $$\boldsymbol{u}(\boldsymbol{x};\mu)=\mu\bigg \{20(x_1-2.5)(5-x_1),0\bigg \} \quad \forall \boldsymbol{x}=(x_0,x_1) \in \Omega$$ 

This problem is characterized by one parameter $\mu$, which characterizes the kinematic viscosity of the fluid. The range of $\mu$ is the following $$\mu \in \mathbb{P}=[0.5,2.0].$$ 

In order to obtain a faster approximation of the problem and an efficient reconstruction of the bifurcation diagram, we pursue a model reduction by means of a POD-Galerkin reduced order method.

In [ ]:
# Install FEniCS
try:
    import dolfin
except ImportError:
    !wget "https://fem-on-colab.github.io/releases/fenics-install-real.sh" -O "/tmp/fenics-install.sh" && bash "/tmp/fenics-install.sh"
    import dolfin

In [ ]:
# Install RBniCS
try:
    import rbnics
except ImportError:
    !pip3 install git+https://github.com/RBniCS/RBniCS.git
    import rbnics
import rbnics.utils.config
assert "dolfin" in rbnics.utils.config.config.get("backends", "required backends")

In [ ]:
# Download data files
!mkdir -p data
!mkdir -p utils
![ -f data/channel.xml ] || wget https://github.com/RBniCS/RBniCS/raw/master/tutorials/20_coanda_effect/data/channel.xml -O data/channel.xml
![ -f data/channel_facet_region.xml ] || wget https://github.com/RBniCS/RBniCS/raw/master/tutorials/20_coanda_effect/data/channel_facet_region.xml -O data/channel_facet_region.xml
![ -f data/channel_physical_region.xml ] || wget https://github.com/RBniCS/RBniCS/raw/master/tutorials/20_coanda_effect/data/channel_physical_region.xml -O data/channel_physical_region.xml
![ -f utils/__init__.py ] || wget https://github.com/RBniCS/RBniCS/raw/master/tutorials/20_coanda_effect/utils/__init__.py -O utils/__init__.py
![ -f utils/error_analysis.py ] || wget https://github.com/RBniCS/RBniCS/raw/master/tutorials/20_coanda_effect/utils/error_analysis.py -O utils/error_analysis.py
![ -f utils/speedup_analysis.py ] || wget https://github.com/RBniCS/RBniCS/raw/master/tutorials/20_coanda_effect/utils/speedup_analysis.py -O utils/speedup_analysis.py

### 2. Parametrized formulation

Let $\boldsymbol{u}(\mu)$ be the velocity vector and $p(\mu)$ be the pressure in the domain $\Omega$.

We will directly provide a weak formulation for this problem: for a given parameter $\mu \in \mathbb{P},$ find $u(\mu) \in \mathbb{V}(\mu), \; p \in\mathbb{M}$ such that
    

$$
\begin{cases}
    \mu \int_{\Omega} \nabla \boldsymbol{u} : \nabla \boldsymbol{v} \ d\Omega + \int_{\Omega} [(\boldsymbol{u} \cdot \nabla) \boldsymbol{u}] \cdot \boldsymbol{v} \ d\Omega - \int_{\Omega} p \nabla \cdot \boldsymbol{v} \ d\Omega = \int_{\Omega} \boldsymbol{f} \cdot \boldsymbol{v} \ d\Omega, \quad \forall \boldsymbol{v} \in\mathbb{V},  \\
    \int_{\Omega} q \nabla \cdot \boldsymbol{u} \ d\Omega = 0, \quad \forall q \in\mathbb{M}
\end{cases}
$$
 
where

* $\mu$ represents kinematic viscosity
* the functional space $\mathbb{V}(\mu)$ is defined as $\mathbb{V}=[H^1_{\Gamma_{wall}}(\Omega)]^2$
* the functional space $\mathbb{M}(\mu)$ is defined as $\mathbb{M}=L^2(\Omega)$


Since this problem utilizes mixed finite element discretization with the velocity and pressure as solution variables, the inf-sup condition is necessary for the well posedness of this problem. Thus, the supremizer operator $T^{\mu}: \mathbb{M}_h \rightarrow \mathbb{V}_h$ will be used.

In [ ]:
from dolfin import *
from rbnics import *
from rbnics.backends.online import OnlineFunction
from rbnics.backends import assign
import numpy as np
import matplotlib.pyplot as plt
from utils import *

### 3. Affine Decomposition 



In [ ]:
@ExactParametrizedFunctions()
class Coanda(NavierStokesProblem):

    # Default initialization of members
    def __init__(self, V, **kwargs):
        # Call the standard initialization
        NavierStokesProblem.__init__(self, V, **kwargs)
        # ... and also store FEniCS data structures for assembly
        assert "subdomains" in kwargs
        assert "boundaries" in kwargs
        self.subdomains, self.boundaries = kwargs["subdomains"], kwargs["boundaries"]
        dup = TrialFunction(V)
        (self.du, self.dp) = split(dup)
        (self.u, _) = split(self._solution)
        vq = TestFunction(V)
        (self.v, self.q) = split(vq)
        self.dx = Measure("dx")(subdomain_data=self.subdomains)
        self.ds = Measure("ds")(subdomain_data=self.boundaries)
        self.inlet = Expression(("20*(x[1] - 2.5)*(5 - x[1])", "0."), degree=2)
        self.f = Constant((0.0, 0.0))
        self.g = Constant(0.0)
        self._solution_prev = Function(V)
        # Customize nonlinear solver parameters
        self._nonlinear_solver_parameters.update({
            "linear_solver": "umfpack",
            "maximum_iterations": 20,
            "report": True
        })

    # Return custom problem name
    def name(self):
        return "CoandaEffect"

    # Return theta multiplicative terms of the affine expansion of the problem.
    @compute_theta_for_derivatives
    @compute_theta_for_supremizers
    def compute_theta(self, term):
        mu = self.mu
        if term == "a":
            theta_a0 = mu[0]
            return (theta_a0,)
        elif term in ("b", "bt"):
            theta_b0 = 1.
            return (theta_b0,)
        elif term == "c":
            theta_c0 = 1.
            return (theta_c0,)
        elif term == "f":
            theta_f0 = 1.
            return (theta_f0,)
        elif term == "g":
            theta_g0 = 1.
            return (theta_g0,)
        elif term == "dirichlet_bc_u":
            theta_bc00 = 1.
            return (theta_bc00,)
        else:
            raise ValueError("Invalid term for compute_theta().")

    # Return forms resulting from the discretization of the affine expansion of the problem operators.
    @assemble_operator_for_derivatives
    @assemble_operator_for_supremizers
    def assemble_operator(self, term):
        dx = self.dx
        if term == "a":
            u = self.du
            v = self.v
            a0 = inner(grad(u), grad(v))*dx
            return (a0,)
        elif term == "b":
            u = self.du
            q = self.q
            b0 = - q*div(u)*dx
            return (b0,)
        elif term == "bt":
            p = self.dp
            v = self.v
            bt0 = - p*div(v)*dx
            return (bt0,)
        elif term == "c":
            u = self.u
            v = self.v
            c0 = inner(grad(u)*u, v)*dx
            return (c0,)
        elif term == "f":
            v = self.v
            f0 = inner(self.f, v)*dx
            return (f0,)
        elif term == "g":
            q = self.q
            g0 = self.g*q*dx
            return (g0,)
        elif term == "dirichlet_bc_u":
            bc0 = [DirichletBC(self.V.sub(0), self.inlet, self.boundaries, 1),
                   DirichletBC(self.V.sub(0), Constant((0.0, 0.0)), self.boundaries, 2)]
            return (bc0,)
        elif term == "inner_product_u":
            u = self.du
            v = self.v
            x0 = inner(grad(u), grad(v))*dx
            return (x0,)
        elif term == "inner_product_p":
            p = self.dp
            q = self.q
            x0 = inner(p, q)*dx
            return (x0,)
        else:
            raise ValueError("Invalid term for assemble_operator().")

    # Simple continuation method to reconstruct a branch of the bifurcation diagram
    def _solve(self, **kwargs):
        assign(self._solution, self._solution_prev)
        NavierStokesProblem._solve(self, **kwargs)
        assign(self._solution_prev, self._solution)

# Customize the resulting reduced problem to enable simple continuation at reduced level
@CustomizeReducedProblemFor(NavierStokesProblem)
def CustomizeReducedNavierStokes(ReducedNavierStokes_Base):
    class ReducedNavierStokes(ReducedNavierStokes_Base):
        def __init__(self, truth_problem, **kwargs):
            ReducedNavierStokes_Base.__init__(self, truth_problem, **kwargs)
            self._solution_prev = None
            self._nonlinear_solver_parameters.update({
                "report": True,
                "line_search": "wolfe",
                "maximum_iterations": 20
            })
            self.flag = False

        def _solve(self, N, **kwargs):
            if self.flag:
                assign(self._solution, self._solution_prev)
            ReducedNavierStokes_Base._solve(self, N, **kwargs)
            self._solution_prev = OnlineFunction(N)
            assign(self._solution_prev, self._solution)
            self.flag = True
    return ReducedNavierStokes

## 4. Main program

### 4.1. Read the mesh for this problem
The mesh was generated by the [data/generate_mesh.ipynb](data/generate_mesh.ipynb) notebook.

In [ ]:
mesh = Mesh("data/channel.xml")
subdomains = MeshFunction("size_t", mesh, "data/channel_physical_region.xml")
boundaries = MeshFunction("size_t", mesh, "data/channel_facet_region.xml")

### 4.2. Create Finite Element Space (Taylor-Hood P2-P1)

In [ ]:
element_u = VectorElement("Lagrange", mesh.ufl_cell(), 2)
element_p = FiniteElement("Lagrange", mesh.ufl_cell(), 1)
element = MixedElement(element_u, element_p)
V = FunctionSpace(mesh, element, components=[["u", "s"], "p"])

### 4.3. Allocate an object of the NavierStokes class

In [ ]:
problem = Coanda(V, subdomains=subdomains, boundaries=boundaries)
mu_range = [(2., 0.5)]
problem.set_mu_range(mu_range)

### 4.4. Prepare reduction with a POD-Galerkin method

In [ ]:
reduction_method = PODGalerkin(problem)
reduction_method.set_Nmax(20)
reduction_method.set_tolerance(1e-8)

### 4.5. Perform the offline phase

In [ ]:
lifting_mu = (2.0,)
problem.set_mu(lifting_mu)
reduction_method.initialize_training_set(51, sampling=EquispacedDistribution())
reduced_problem = reduction_method.offline()

### 4.6. Perform the online reconstruction of the bifurcation diagram

In [ ]:
# Quantities for the bifurcation analysis
mu_start_bif = 2.
mu_end_bif = 0.5
mu_num_bif = 101
mu_range_bif, mu_step_bif = np.linspace(mu_start_bif, mu_end_bif, mu_num_bif, retstep=True)

# Quantities for the bifurcation diagram
hf_output = list()
rb_output = list()
coor = mesh.coordinates()

bc = [DirichletBC(V.sub(0), problem.inlet, boundaries, 1), DirichletBC(V.sub(0), Constant((0.0, 0.0)), boundaries, 2)]

In [ ]:
for (i,mu) in enumerate(mu_range_bif):

    online_mu = (mu,)
    problem.set_mu(online_mu)
    solution = problem.solve()
    problem.export_solution("CoandaEffect", "online_solution_hf", suffix=i)
    (y_hf, p_hf) = split(solution)
    hf_output.append(solution.sub(0)(coor[72])[1])

    reduced_problem.set_mu(online_mu)
    reduced_solution = reduced_problem.solve()
    Z = reduced_problem.basis_functions*reduced_solution
    reduced_problem.export_solution("CoandaEffect", "online_solution_ro", suffix=i)
    (y_ro, p_ro) = split(Z)
    rb_output.append(Z.sub(0)(coor[72])[1])

In [ ]:
plot(solution.sub(0).sub(0))

In [ ]:
plot(solution.sub(1))

In [ ]:
# 7. Perform error and speedup analysis
reduction_method.initialize_testing_set(151, sampling=EquispacedDistribution())
N_max = min(reduced_problem.N.values())

error_analysis_coanda(reduction_method, N_max, filename="error_analysis")
speedup_analysis_coanda(reduction_method, N_max, filename="speedup_analysis")

### 4.7. Plot bifurcation diagrams

In [ ]:
plt.figure("Bifurcation analysis")
plt.plot(mu_range_bif, hf_output, "-r", linewidth=2, label = "HF output")
plt.plot(mu_range_bif, rb_output, "--b", linewidth=2, label = "RB output")
plt.xlabel('$\\mu$')
plt.ylabel('$v_x(11.62, 3.75)$')
plt.title("Bifurcation Diagram")
plt.legend()
plt.grid(True)
plt.show()